In [254]:
import matplotlib.pyplot as plt
import pandas as pd

full_pumpinks = pd.read_csv("../data/US-pumpkins.csv")
full_pumpinks.head()
full_pumpinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City Name        1757 non-null   object 
 1   Type             45 non-null     object 
 2   Package          1757 non-null   object 
 3   Variety          1752 non-null   object 
 4   Sub Variety      296 non-null    object 
 5   Grade            0 non-null      float64
 6   Date             1757 non-null   object 
 7   Low Price        1757 non-null   float64
 8   High Price       1757 non-null   float64
 9   Mostly Low       1654 non-null   float64
 10  Mostly High      1654 non-null   float64
 11  Origin           1754 non-null   object 
 12  Origin District  131 non-null    object 
 13  Item Size        1478 non-null   object 
 14  Color            1141 non-null   object 
 15  Environment      0 non-null      float64
 16  Unit of Sale     162 non-null    object 
 17  Quality       

In [255]:
col_selected = ['City Name', 'Package', 'Date', 'Low Price', 'High Price', 'Repack', 'Color', 'Item Size', 'Origin']
pumpkins = full_pumpinks[col_selected]
pumpkins.dropna(inplace=True)
pumpkins.head()

C:\Users\31225\AppData\Local\Temp\ipykernel_21136\4272244492.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pumpkins.dropna(inplace=True)


,City Name,Package,Date,Low Price,High Price,Repack,Color,Item Size,Origin
2,BALTIMORE,24 inch bins,9/24/16,160.0,160.0,N,ORANGE,med,DELAWARE
3,BALTIMORE,24 inch bins,9/24/16,160.0,160.0,N,ORANGE,med,VIRGINIA
4,BALTIMORE,24 inch bins,11/5/16,90.0,100.0,N,ORANGE,lge,MARYLAND
5,BALTIMORE,24 inch bins,11/12/16,90.0,100.0,N,ORANGE,lge,MARYLAND
6,BALTIMORE,36 inch bins,9/24/16,160.0,170.0,N,ORANGE,med,MARYLAND


In [256]:
# 日期 价格处理
month = pd.DatetimeIndex(pumpkins['Date']).month
price = (pumpkins['Low Price'] + pumpkins['High Price']) / 2

new_pumpkins = pd.DataFrame( {'Month': month, 'Price': price, 'City Name': pumpkins['City Name'], 
                              'Package': pumpkins['Package'], 'Repack': pumpkins['Repack'], 
                              'Color': pumpkins['Color'], 'Item Size': pumpkins['Item Size'], 
                              'Origin': pumpkins['Origin']})
new_pumpkins

,Month,Price,City Name,Package,Repack,Color,Item Size,Origin
2,9,160.0,BALTIMORE,24 inch bins,N,ORANGE,med,DELAWARE
3,9,160.0,BALTIMORE,24 inch bins,N,ORANGE,med,VIRGINIA
4,11,95.0,BALTIMORE,24 inch bins,N,ORANGE,lge,MARYLAND
5,11,95.0,BALTIMORE,24 inch bins,N,ORANGE,lge,MARYLAND
6,9,165.0,BALTIMORE,36 inch bins,N,ORANGE,med,MARYLAND
...,...,...,...,...,...,...,...,...
1694,9,150.0,ST. LOUIS,24 inch bins,N,WHITE,xlge,ILLINOIS
1695,9,150.0,ST. LOUIS,24 inch bins,N,WHITE,xlge,ILLINOIS
1696,9,150.0,ST. LOUIS,24 inch bins,N,WHITE,xlge,ILLINOIS
1697,9,150.0,ST. LOUIS,24 inch bins,N,WHITE,xlge,ILLINOIS


In [257]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

label_col = ['Origin', 'Color']
label_encoder = LabelEncoder()
new_pumpkins['Origin_encoded'] = label_encoder.fit_transform(new_pumpkins.Origin)
new_pumpkins['Color_encoded'] = label_encoder.fit_transform(new_pumpkins.Color)
# new_pumpkins = new_pumpkins.drop('Origin', axis=1)

onehot_col = ['Package', 'Repack', 'City Name']
onehot_encoder = OneHotEncoder(sparse_output=False)

ordinal_col = ['Item Size']
ordinal_encoder = OrdinalEncoder(categories = [['sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo']])

ct = ColumnTransformer([
    ('onehot', onehot_encoder, onehot_col),
    ('ordinal' , ordinal_encoder, ordinal_col),
])
ct.set_output(transform='pandas')
encode_pumpkins = ct.fit_transform(new_pumpkins)
encode_pumpkins = pd.concat([new_pumpkins, encode_pumpkins], axis=1)
encode_pumpkins = encode_pumpkins.drop(label_col, axis=1).drop(onehot_col, axis = 1).drop(ordinal_col, axis=1)
encode_pumpkins.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 991 entries, 2 to 1698
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Month                                 991 non-null    int64  
 1   Price                                 991 non-null    float64
 2   Origin_encoded                        991 non-null    int32  
 3   Color_encoded                         991 non-null    int32  
 4   onehot__Package_1 1/9 bushel cartons  991 non-null    float64
 5   onehot__Package_1 1/9 bushel crates   991 non-null    float64
 6   onehot__Package_1/2 bushel cartons    991 non-null    float64
 7   onehot__Package_24 inch bins          991 non-null    float64
 8   onehot__Package_36 inch bins          991 non-null    float64
 9   onehot__Package_bins                  991 non-null    float64
 10  onehot__Package_bushel cartons        991 non-null    float64
 11  onehot__Repack_N  

In [265]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

x_train, x_test, y_train, y_test = train_test_split(encode_pumpkins[encode_pumpkins.columns.difference(['Color_encoded', 'Price'])], encode_pumpkins['Color_encoded'], test_size=0.2, random_state=0)
model = LogisticRegression()
model.fit(x_train, y_train)
pred = model.predict(x_test)
model.score(x_train, y_train)

D:\work\language\miniconda\envs\pytorch_39\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8787878787878788

In [267]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92       166
           1       0.65      0.33      0.44        33

    accuracy                           0.86       199
   macro avg       0.76      0.65      0.68       199
weighted avg       0.84      0.86      0.84       199
